In [1]:
import os
import random

import torch
from torch.utils.data import DataLoader
from miditok import REMI, TokenizerConfig, MIDITokenizer, TokSequence
from miditok.pytorch_data import DatasetMIDI, DatasetJSON, DataCollator, split_midis_for_training
from miditok.data_augmentation import augment_midi_dataset
from miditok.utils import get_midi_programs
from pathlib import Path
from symusic import Score
import wandb
from tqdm import tqdm

from transformers.models.opt.modeling_opt import OPTForCausalLM

/home/lklimkiewicz/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Tokenizer

In [ ]:
midi_paths = list(Path("/home/lklimkiewicz/priv/midi/data").glob("**/*.mid"))

In [3]:
tokenizer_config = TokenizerConfig(num_velocities=16, use_chords=True, use_programs=True)
tokenizer = REMI(tokenizer_config)

In [ ]:
random.shuffle(midi_paths)
tokenizer.learn_bpe(vocab_size=30000, files_paths=midi_paths[:1000])

In [5]:
# saving
tokenizer.save_pretrained('../logs/tokenizer2')

In [2]:
# loading
tokenizer = MIDITokenizer.from_pretrained('../logs/tokenizer2')

config.json not found in /home/lklimkiewicz/priv/midi/logs/tokenizer2


## Model

In [3]:
from transformers import AutoConfig, AutoModel, AutoModelForCausalLM

In [4]:
config = AutoConfig.from_pretrained(
    'facebook/opt-125m',
    bos_token_id=tokenizer['BOS_None'],
    eos_token_id=tokenizer['EOS_None'],
    pad_token_id=tokenizer['PAD_None'],
    vocab_size=len(tokenizer),
    prefix=None,
    max_length=1024,
    do_sample=True,
)

In [5]:
model = AutoModelForCausalLM.from_config(config)

## Split dataset

In [6]:
midi_paths = list(Path("/home/lklimkiewicz/priv/midi/data").glob("**/*.mid"))

In [9]:
def filter_dataset(paths, val_fun):
    correct = []
    for path in tqdm(paths):
        try:
            midi = Score(path)
            if val_fun(midi):
                correct.append(path)
        except:
            os.remove(path)
    return correct

print('Initial count:', len(midi_paths))

def midi_valid(midi) -> bool:
    if midi.note_num() < 50 or len(midi.time_signatures) == 0 or len(midi.tempos) == 0:
        return False
    
    for time_sig in midi.time_signatures:
        if time_sig.denominator == 0 or time_sig.numerator == 0:
            return False
    
    return True

midi_paths = filter_dataset(midi_paths, midi_valid)

print('Filtered count:', len(midi_paths))

Initial count: 1067913


100%|██████████| 1067913/1067913 [13:06<00:00, 1357.29it/s]


Filtered count: 766717


In [ ]:
split_midis_for_training(
    files_paths=midi_paths,
    tokenizer=tokenizer,
    save_dir=Path('./chunks_for_training'),
    max_seq_len=1024,
)

## Load dataset

In [6]:
midi_paths = list(Path("/home/lklimkiewicz/priv/midi/data_prim/chunks_for_training_2").glob("**/*.mid"))

In [ ]:
print('Initial count:', len(midi_paths))

def filter_dataset(paths, val_fun):
    correct = []
    for path in tqdm(paths):
        try:
            midi = Score(path)
            if val_fun(midi):
                correct.append(path)
        except:
            pass
    return correct

def midi_valid(midi) -> bool:
    if midi.note_num() < 50 or len(midi.time_signatures) == 0 or len(midi.tempos) == 0:
        return False
    
    for time_sig in midi.time_signatures:
        if time_sig.denominator == 0 or time_sig.numerator == 0:
            return False
    
    return True

midi_paths = filter_dataset(midi_paths, midi_valid)

print('Filtered count:', len(midi_paths))

In [7]:
dataset = DatasetMIDI(
    files_paths=midi_paths,
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)

collator = DataCollator(
    tokenizer["PAD_None"],
    copy_inputs_as_labels=True,
    shift_labels=True,
)

## Train

In [8]:
from transformers import Trainer, TrainingArguments, TrainerCallback, TrainerState, TrainerControl

In [9]:
class MidiGenerationCallback(TrainerCallback):
    def on_step_end(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        if state.global_step % 500 == 0:
            
            input = torch.tensor([[tokenizer['BOS_None']]], device=model.device)
            midi = model.generate(input, max_new_tokens=1024)
            generated_ts = TokSequence(ids=midi.tolist()[0], ids_bpe_encoded=True)
            generated_score = tokenizer(generated_ts)
            generated_score.dump_midi(f'outputs/v1/step-{state.global_step}.mid')

In [ ]:
training_args = TrainingArguments(
    output_dir="../logs/tmp/output",
    per_device_train_batch_size=4,
    report_to="wandb",
    bf16=True,
    dataloader_num_workers=24,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    lr_scheduler_type="cosine",
    warmup_steps=2000,
    save_steps=1000,
    save_total_limit=5,
    num_train_epochs=1,
    label_smoothing_factor=0.2,
    torch_compile=True,
    deepspeed=True
)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=collator,
    tokenizer=tokenizer,
    # callbacks=[MidiGenerationCallback()]
)

In [12]:
os.environ["WANDB_PROJECT"] = "midi"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

fp32: 2.2 it/s
bf16: 3.3 it/s
bf16 + compile: 3.4 it/s

In [13]:
trainer.train()

/home/lklimkiewicz/miniconda3/envs/midi2/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: weights-and-biases (klima7-team). Use `wandb login --relogin` to force relogin


  0%|          | 0/974907 [00:00<?, ?it/s]/home/lklimkiewicz/miniconda3/envs/midi2/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
  0%|          | 19/974907 [01:04<139:28:12,  1.94it/s][2024-04-22 00:30:59,670] torch._dynamo.convert_frame: [WARNING] torch._dynamo hit config.cache_size_limit (8)
[2024-04-22 00:30:59,670] torch._dynamo.convert_frame: [WARNING]    function: '_prepare_4d_causal_attention_mask' (/home/lklimkiewicz/.local/lib/python3.10/site-packages/transformers/modeling_attn_mask_utils.py:278)
[2024-04-22 00:30:59,670] torch._dynamo.convert_frame: [WARNING]    l

{'loss': 10.4085, 'learning_rate': 2.5e-06, 'epoch': 0.0}


  0%|          | 200/974907 [02:26<122:25:11,  2.21it/s]

{'loss': 10.2045, 'learning_rate': 5e-06, 'epoch': 0.0}


  0%|          | 218/974907 [02:34<124:51:32,  2.17it/s]

KeyboardInterrupt: 

In [ ]:
model.save_pretrained('../logs/model2', tokenizer=tokenizer)

In [16]:
tokenizer.save_pretrained('../logs/tokenizer2')

In [ ]:
wandb.finish()